# Visualization Tool for Planet Imagery to QAQC & Analyze NDVI SITS
## Binary Mask Visualizer Isolated

This notebooks focuses on providing the tools necessary to QAQC Planet imagery and to visualization the changes in NDVI over a set or subset of images for a given Area of Interest (AOI). The intention is to be able to determine if a given AOI is increasing or decreasing in vegetation (i.e. greenness) and the general hotspots this may or may not be occurring. This notebook is intended to be used with other preprocessing tools and assumes that the user has a single directory containing analysis-ready NDVI rasters (i.e. single-band -1 to 1 *.tiffs).

### This notebook is only for deriving binary thresholding masks of NDVI and evaluating the associated thresholding algorithms.

#### Import Libary Setup

The libraries below must be installed in the Anaconda environment that the user is running this notebook from.

In [ ]:
# Imported Python standard libraries
import os
import sys
import datetime
from functools import partial
# Imported Python data manipulation libraries
import numpy as np
import rasterio
from skimage.filters import (threshold_minimum, threshold_li, threshold_isodata,
                             threshold_mean, threshold_otsu, try_all_threshold)
# Imported Python visualization libraries
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, Label
from IPython.display import display, clear_output

#### User Provided Inputs Setup

Please specify the input folder directory containing all the Planet imagery to be analyzed as NDVI rasters. The absolute folder pathname is recommended along with the use of r" " to reverse the "\\" to "/" automatically.

In [ ]:
# input folder directory
planet_input_dir = r"C:\Users\zleady\Desktop\ST_1867\LC\lcr_active_20200621\poh_final_NDVI"

In [ ]:
def CreateImgPathList(img_folder):
    """ Creates a Python List of Image Pathnames
    """
    img_path_lst = [os.path.join(img_folder, f) 
                    for f in os.listdir(img_folder)
                    if f.endswith(".tif")]
    return img_path_lst

Invoke the CreateImgPathList function on the planet_input_dir.  
Displays to the user the first and last image/raster pathname and the total number of rasters in the list.

In [ ]:
planet_img_path_lst = CreateImgPathList(planet_input_dir)
# Display output checks
print("First Pathname: ", planet_img_path_lst[0])
print("Last Pathname: ", planet_img_path_lst[-1])
print("Total Number of Pathnames: ", len(planet_img_path_lst))

### Analysis of NDVI Images for Binary Mask Creation

#### Read in NDVI Image Data

Read-in the NDVI images/rasters into a Python dictionary called "ndvi_img_dict". Each image has a date key in the format "YYYYMMDD", which is paired to another dictionary containing the keys "path", "original_array", "histo_array", and "display_array". The histo_array is modified to not contain any NaN's which are used for pixels with missing data by numpy when the image is read into a numpy array via the rasterio library. The display_array converts the NaN's into 0 for easier displaying with the matplotlib libraries "imshow()" function. This code block also ensures that each image is the same number of pixels on the x-axis and y-axis and filters images into the incorrect_shape_lst that are not the same shape.

In [ ]:
# initialize image dictionary
ndvi_img_dict = {}
# initialize list for images with the wrong shape
incorrect_shape_lst = []

In [ ]:
for p in planet_img_path_lst:    
    with rasterio.open(p) as ds:
        temp_arr = ds.read(1)
        temp_dict = {"path": p,
                     "original_array": temp_arr,
                     "histo_array": temp_arr[~np.isnan(temp_arr)],
                     "display_array": np.nan_to_num(temp_arr)}
        date_id = os.path.basename(p).split("_")[0]
        if p == planet_img_path_lst[0]:
            x0_shape = temp_arr.shape[0]
            y0_shape = temp_arr.shape[1]
            print("First x0_shape: {}, y0_shape: {}".format(x0_shape, y0_shape))
        if temp_arr.shape[0] == x0_shape and temp_arr.shape[1] == y0_shape:
            ndvi_img_dict[date_id] = temp_dict
        else:
            incorrect_shape_lst.append([date_id, temp_arr.shape])
    ds.close()

In [ ]:
img_key_lst = list(ndvi_img_dict.keys())
print("Images found to have an inconsistent shape [date, (array shape)]: ")
print(incorrect_shape_lst)
print("Started with {} Pathnames".format(len(planet_img_path_lst)))
print("Added {} to NDVI Image Dictionary".format(len(img_key_lst)))
print("Check that each image key has 4 keys: ",
      ndvi_img_dict.get("{}".format(img_key_lst[0])).keys())

### Establish Binary Thresholding Method and Array Data

An NDVI image is a "derived" band and ranges between -1 to 1 with 1 signifying high vegetation or "greenness". Binary thresholding methods (algorithms) can be used to determine a seperation from vegetation and everything else. This is not the primary goal of NDVI tracking but can serve as a good QAQC check for creating a boundary around what might be an actual change in NDVI vs. a spurious or insignificant change in NDVI. Thus the binary threshold array should be viewed as a loose boundary of what to focus on versus the rest of the image.

In [ ]:
# dictionary of binary threshold functions bound to string identifiers
# useful for looping functions
threshold_dict = {"iso": threshold_isodata, "li": threshold_li,
                  "mean": threshold_mean, "otsu": threshold_otsu,
                  "min": threshold_minimum}
# initialize list for dates that binary thresholding failed on for data/algo reasons
failed_threshold_lst = []

# binary thresholding loop
print("Starting thresholding loop")
for date in list(ndvi_img_dict.keys()):
    temp_arr = ndvi_img_dict.get(date).get("histo_array")
    print("Processing: {}".format(date))
    try:
        for k in list(threshold_dict.keys()):
            temp_threshold_val = threshold_dict.get(k)(temp_arr)
            temp_binary_array = ndvi_img_dict.get(date).get("display_array") > temp_threshold_val
            ndvi_img_dict.get(date)["{}_value".format(k)] = temp_threshold_val
            ndvi_img_dict.get(date)["{}_binary_array".format(k)] = temp_binary_array
    except:
        failed_threshold_lst.append([date])
        del ndvi_img_dict[date]
print("Finished")

In [ ]:
print("Failed to threshold the following dates: ")
print(failed_threshold_lst)
print("Started with {} number of images for thresholding".format(len(img_key_lst)))
print("Thresholded {0} images. {0} images available for analysis.".format(len(list(ndvi_img_dict.keys()))))
print("\n")
print("Check the new keys for each image: ")
print(ndvi_img_dict.get("{}".format(list(ndvi_img_dict.keys())[0])).keys())

### Visually Compare Thresholding Method Performance

Each Binary Thresholding Method (algorithm) works in a slightly different way to automatically determine the seperation on the NDVI image histogram for vegetation and non-vegetation. The function and visualizer below allow the user to determine over there set or subset of images which thresholding method to use; typically, depending on how conservative the user chooses to be.

In [ ]:
def show_compare_thresholds(img_dict):
    """ Plotting function for binary threshold comparison
    """
    global threshold_dict # remove in the future, a shortcut for ipywidgets observer
    plt.figure(figsize=(24, 48))
    plt.subplot(6, 2, 1)
    plt.imshow(img_dict.get("display_array"), cmap=plt.cm.gray)
    plt.title("Original NDVI Image")
    plt.subplot(6, 2, 2)
    plt.hist(img_dict.get("histo_array"), bins=256)
    plt.title("Original NDVI Histogram")
    subplot_spacer = 2
    for indx, k in enumerate(list(threshold_dict.keys())):
        plt.subplot(6, 2, indx+1+subplot_spacer)
        plt.imshow(img_dict.get("{}_binary_array".format(k)), cmap=plt.cm.gray)
        plt.title('{} Binary NDVI Image'.format(k.upper()))
        plt.subplot(6, 2, indx+2+subplot_spacer)
        plt.hist(img_dict.get("histo_array"), bins=256)
        plt.axvline(img_dict.get("{}_value".format(k)), color="r")
        plt.title("{} Thresholded Histogram Value={}".format(k.upper(), round(float(img_dict.get("{}_value".format(k))), 4)))
        subplot_spacer += 1
    plt.show()

A Visualizer for the Binary Threshold Comparison using ipywidgets. Change the dropdown menu date to check different images.

In [ ]:
threshold_date_options_lst = sorted(list(ndvi_img_dict.keys()))
threshold_date_dropdown = widgets.Dropdown(options=threshold_date_options_lst,
                                           value=threshold_date_options_lst[0],
                                           description='Image Date:',)


def threshold_date_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        display(threshold_date_dropdown)
        print(change['old'], change['new'])
        show_compare_thresholds(ndvi_img_dict.get("{}".format(change['new'])))

threshold_date_dropdown.observe(threshold_date_on_change)
display(threshold_date_dropdown)

#### If a user is unsure on the binary thresholding algorithm choice OTSU is the literature default.  
#### Zack's personal recommendation is normally threshold_minimum ("min") as a much more conservative default.

Please set a binary thresholding algorithm below.

In [ ]:
# options
print(threshold_dict.keys())

In [ ]:
# choosen binary thresholding algorithm
binary_algo = "otsu"

### Export Binary Thresholded NDVI Imagery

The following cells export the binary thresholded NDVI images as "*.tif" rasters using their original NDVI metadata.
  
The user can view the rasters in their preferred GIS program (i.e. ArcGIS, QGIS, etc.)  
OR  
the rasters can be read by a Python library such as rasterio back into numpy arrays.

In [ ]:
output_dir = os.path.join(os.path.dirname(planet_input_dir), "poh_final_NDVI_Binary_{}".format(binary_algo.upper()))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
print("All binary rasters will be written too: ")
print(output_dir)

In [ ]:
for date in list(ndvi_img_dict.keys()):
    retrieve_path = ndvi_img_dict.get(date).get("path")
    assert os.path.exists(retrieve_path)
    with rasterio.open(retrieve_path) as orig_dst:
        kwargs = orig_dst.meta
        kwargs.update(dtype=rasterio.float32, count=1)        
        output_path = os.path.join(output_dir,
                                   "{}_binary_{}.tif".format(date, binary_algo))
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write_band(1, ndvi_img_dict.get(date).get("{}_binary_array".format(binary_algo)).astype(rasterio.float32))
    print("Wrote {} binary array using {} algo".format(date, binary_algo))

En Fin.